In [1]:
from multiprocessing import Pool
from tqdm import tqdm
from spacy.lang.en import English
import gensim.corpora as corpora
from gensim.models import LdaMulticore
import multiprocessing as mp

import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
from nltk.corpus import wordnet as wn
#nltk.download('stopwords')
#nltk.download('wordnet')
# spacy for lemmatization
import spacy
from spacy.lang.en import English
parser = English()

# Plotting tools

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [20]:
import json
from sklearn.feature_extraction.text import CountVectorizer
def tokenize(doc):
    tokens = doc.split(" ")
    tokens = [word for word in tokens if len(word.strip()) > 0]
    return tokens


parser = English()

print("Loading tech corpus")
with open(("../data/tech_review_sent_corpus.pkl"), "rb") as f:
        tech_review_corpus = pickle.load(f)
reviews = pd.DataFrame(tech_review_corpus).review.tolist()

print("Total workers:", num_cpus)

print("Tokenize the corpus")

with open(( "../data/stop_words.json"), "r") as f:
    stop_words = json.load(f)

vectorizer = CountVectorizer(
        min_df=3, max_df=.90, tokenizer=tokenize, stop_words=stop_words, ngram_range=(1, 2))


Loading tech corpus
Total workers: 7
Tokenize the corpus


In [21]:
X = vectorizer.fit_transform(reviews)

In [44]:
from gensim.corpora.dictionary import Dictionary
def vect2gensim(vectorizer, dtmatrix):
     # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = gensim.matutils.Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = Dictionary.from_corpus(corpus_vect_gensim,
        id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))

    return (corpus_vect_gensim, dictionary)
corpus, id2word = vect2gensim(vectorizer, X)

In [46]:
corpus[0]

[(18082, 1),
 (32916, 1),
 (5538, 1),
 (26940, 1),
 (10324, 1),
 (18300, 1),
 (32959, 1),
 (5578, 1),
 (26957, 1)]

In [53]:
final_lda_model = LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=500,
                                           workers = 4,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.61,
                                           per_word_topics=True)

In [54]:
# Select the model and print the topics
optimal_model = final_lda_model
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=30))

[(0,
  '0.067*"work" + 0.047*"good" + 0.037*"life" + 0.029*"balance" + 0.026*"work '
  'life" + 0.025*"life balance" + 0.020*"co" + 0.018*"benefit" + 0.017*"co '
  'work" + 0.013*"salary" + 0.011*"flexible" + 0.008*"pay" + 0.008*"good work" '
  '+ 0.007*"project" + 0.006*"management" + 0.006*"home" + 0.006*"employee" + '
  '0.006*"le" + 0.006*"compensate" + 0.006*"hour" + 0.005*"work home" + '
  '0.005*"job" + 0.005*"depend" + 0.004*"also" + 0.004*"hike" + '
  '0.004*"provide" + 0.004*"work hour" + 0.004*"pressure" + 0.004*"flexible '
  'work" + 0.004*"facility"'),
 (1,
  '0.099*"work" + 0.058*"good" + 0.049*"great" + 0.046*"place" + '
  '0.030*"environment" + 0.023*"culture" + 0.022*"place work" + '
  '0.014*"management" + 0.013*"people" + 0.012*"wa" + 0.012*"good work" + '
  '0.012*"work environment" + 0.011*"friendly" + 0.011*"fun" + 0.011*"work '
  'culture" + 0.010*"experience" + 0.010*"good place" + 0.010*"work place" + '
  '0.009*"great place" + 0.007*"great work" + 0.007*"overa

In [55]:
lda_display = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word, sort_topics=False)
pyLDAvis.display(lda_display)

In [61]:
optimal_model.show_topics(num_words=100,formatted=False)

[(0,
  [('work', 0.06678139),
   ('good', 0.047276776),
   ('life', 0.037353188),
   ('balance', 0.028591584),
   ('work life', 0.026378049),
   ('life balance', 0.024649018),
   ('co', 0.01962423),
   ('benefit', 0.018107124),
   ('co work', 0.016974425),
   ('salary', 0.012877208),
   ('flexible', 0.011009763),
   ('pay', 0.008447378),
   ('good work', 0.008228292),
   ('project', 0.00704908),
   ('management', 0.0064137424),
   ('home', 0.0062602106),
   ('employee', 0.006038227),
   ('le', 0.005905395),
   ('compensate', 0.0055277585),
   ('hour', 0.005521047),
   ('work home', 0.005115034),
   ('job', 0.0049833115),
   ('depend', 0.004712257),
   ('also', 0.004411426),
   ('hike', 0.004093199),
   ('provide', 0.0036814446),
   ('work hour', 0.0036558344),
   ('pressure', 0.003538302),
   ('flexible work', 0.003504889),
   ('facility', 0.0035043368),
   ('average', 0.0033529564),
   ('personal', 0.0033180201),
   ('good benefit', 0.0031407713),
   ('low', 0.0031099806),
   ('balanc

In [104]:
aspect = {}
for t in range(0, optimal_model.num_topics):
    aspect['topic {}: '.format(t)] = [(v[0],str(v[1])) for v in optimal_model.show_topic(t, 50)]

In [105]:
aspect

{'topic 0: ': [('work', '0.06678139'),
  ('good', '0.047276776'),
  ('life', '0.037353188'),
  ('balance', '0.028591584'),
  ('work life', '0.026378049'),
  ('life balance', '0.024649018'),
  ('co', '0.01962423'),
  ('benefit', '0.018107124'),
  ('co work', '0.016974425'),
  ('salary', '0.012877208'),
  ('flexible', '0.011009763'),
  ('pay', '0.008447378'),
  ('good work', '0.008228292'),
  ('project', '0.00704908'),
  ('management', '0.0064137424'),
  ('home', '0.0062602106'),
  ('employee', '0.006038227'),
  ('le', '0.005905395'),
  ('compensate', '0.0055277585'),
  ('hour', '0.005521047'),
  ('work home', '0.005115034'),
  ('job', '0.0049833115'),
  ('depend', '0.004712257'),
  ('also', '0.004411426'),
  ('hike', '0.004093199'),
  ('provide', '0.0036814446'),
  ('work hour', '0.0036558344'),
  ('pressure', '0.003538302'),
  ('flexible work', '0.003504889'),
  ('facility', '0.0035043368'),
  ('average', '0.0033529564'),
  ('personal', '0.0033180201'),
  ('good benefit', '0.0031407713

In [106]:
with open("../results/gensim/gensim-lda-topic-k-8-a-0.01-b-0.61.json", "w") as f:
                json.dump(aspect, f)

In [62]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=reviews):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=final_lda_model, corpus=corpus, texts=reviews)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()

df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

#df_dominant_topic["Review"] = reviews_["review"] 
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.7312,"management, employee, manager, make, people, c...",fluor is a great company with people that care...
1,1,2.0,0.5214,"management, employee, manager, make, people, c...",management care about their employee a well a ...
2,2,7.0,0.6337,"wa, work, job, part, enjoy, hard, time, part j...",fluor wa the first powerhouse epc that went he...
3,3,3.0,0.5853,"work, year, get, would, time, wa, one, love, l...",they send a much work overseas a possible and ...
4,4,2.0,0.5430,"management, employee, manager, make, people, c...",this is a heavy loaded company with management
5,5,5.0,0.5519,"technology, train, work, development, knowledg...",fluor chase the big project and will hire thou...
6,6,3.0,0.9828,"work, year, get, would, time, wa, one, love, l...",in turn they will lay off thousand when it s g...
7,7,2.0,0.9773,"management, employee, manager, make, people, c...",fluor ha the sharpest spike in hiring and layi...
8,8,2.0,0.6094,"management, employee, manager, make, people, c...",the employee is clearly not their great asset
9,9,3.0,0.5535,"work, year, get, would, time, wa, one, love, l...",there is not much that i could add here


In [63]:
with open("../Data/all_reviews.pkl","rb") as f:
    reviews_ = pickle.load(f)
    
job_filter = pd.read_csv("../Data/filter_job_titles.csv")

job_filters = job_filter.clean_job_title.tolist()
idx = (reviews_.language == "en")
reviews_ = reviews_.loc[idx,:]

idx = (reviews_.clean_job_title.isin(job_filters))
reviews_ = reviews_.loc[idx,:].reset_index()

print(reviews_.shape)

(45251, 16)


In [64]:
df_sent = pd.DataFrame(pd.DataFrame(tech_review_corpus).groupby('index'))
df_sent.columns=['Indexes','reviews']
display(df_sent.head(10))

,Indexes,reviews
0,299,index ...
1,1145,index ...
2,1375,index ...
3,1874,index ...
4,2136,index ...
5,2209,index ...
6,2685,index ...
7,3054,index ...
8,3082,index ...
9,3134,index ...


In [65]:
sent_corpus= pd.DataFrame(tech_review_corpus)

In [66]:
sent_corpus

,index,review
0,299,fluor is a great company with people that care...
1,299,management care about their employee a well a ...
2,1145,fluor wa the first powerhouse epc that went he...
3,1145,they send a much work overseas a possible and ...
4,1145,this is a heavy loaded company with management
...,...,...
148441,2587643,most enjoy the teamwork
148442,2587648,work on report make sure everyone is work
148443,2587648,process claim
148444,2587648,i enjoy work with my processor


In [67]:
df_dominant_topic['Index'] = sent_corpus['index']
index_list = list(set(df_dominant_topic["Index"].values))
df_dominant_topic.head(10)
reviews_.review.loc[df_sent[df_sent["Indexes"] == [i for i in index_list]].index]

36301    oracle dba ,i am working here from past 7 yrs ...
Name: review, dtype: object

In [68]:
index_list = list(set(df_dominant_topic["Index"].values))
index_list.sort(reverse=False)
df_dominant_topic_copy = df_dominant_topic.copy()

In [69]:
Review_List = []
for i in index_list:
    if [item for item in (df_dominant_topic.Index.isin([i]).values)]:
        sent_idx = df_dominant_topic['Index'].loc[df_dominant_topic[df_dominant_topic["Index"] == i].index].index
        for x in range(len(sent_idx)):
            Review_List.append(reviews_.review.loc[df_sent[df_sent["Indexes"] == i].index].tolist()[0])
           

In [70]:
df_dominant_topic_copy = df_dominant_topic_copy.assign(Review=Review_List)

In [71]:
df_dominant_topic_copy.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Index,Review
0,0,2.0,0.7312,"management, employee, manager, make, people, c...",fluor is a great company with people that care...,299,Fluor is a great company with people that care...
1,1,2.0,0.5214,"management, employee, manager, make, people, c...",management care about their employee a well a ...,299,Fluor is a great company with people that care...
2,2,7.0,0.6337,"wa, work, job, part, enjoy, hard, time, part j...",fluor wa the first powerhouse epc that went he...,1145,Fluor was the first powerhouse EPC that went h...
3,3,3.0,0.5853,"work, year, get, would, time, wa, one, love, l...",they send a much work overseas a possible and ...,1145,Fluor was the first powerhouse EPC that went h...
4,4,2.0,0.5430,"management, employee, manager, make, people, c...",this is a heavy loaded company with management,1145,Fluor was the first powerhouse EPC that went h...
5,5,5.0,0.5519,"technology, train, work, development, knowledg...",fluor chase the big project and will hire thou...,1145,Fluor was the first powerhouse EPC that went h...
6,6,3.0,0.9828,"work, year, get, would, time, wa, one, love, l...",in turn they will lay off thousand when it s g...,1145,Fluor was the first powerhouse EPC that went h...
7,7,2.0,0.9773,"management, employee, manager, make, people, c...",fluor ha the sharpest spike in hiring and layi...,1145,Fluor was the first powerhouse EPC that went h...
8,8,2.0,0.6094,"management, employee, manager, make, people, c...",the employee is clearly not their great asset,1145,Fluor was the first powerhouse EPC that went h...
9,9,3.0,0.5535,"work, year, get, would, time, wa, one, love, l...",there is not much that i could add here,1375,There is not much that I could add here. I en...


In [77]:
with open("../data/gensim-topic_sent_reviews.pkl","wb") as f:
    pickle.dump(df_dominant_topic_copy,f)

### Sklearn Topic Modeling

In [75]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(
                learning_method="batch",
                random_state=100,
                n_components=7,
                doc_topic_prior=0.01,
                topic_word_prior=0.01,
                n_jobs=-2
            )
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.01,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=-2,
                          perp_tol=0.1, random_state=100, topic_word_prior=0.01,
                          total_samples=1000000.0, verbose=0)

In [76]:
def print_topics(model, vectorizer, top_n=50):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('work', 16325.600778070642), ('good', 7417.687833646841), ('life', 7122.978945092623), ('balance', 5563.009999999923), ('work life', 5138.009999999879), ('life balance', 4801.009999999921), ('co', 3309.175806794377), ('co work', 2859.102910451974), ('project', 2840.167769584037), ('flexible', 2153.7759292643877), ('management', 1979.715081427686), ('good work', 1902.4979737097367), ('time', 1370.3064207372022), ('hour', 1365.5892099899993), ('manager', 1179.9564521685966), ('depend', 1088.6036502730235), ('home', 1069.7372718683), ('work home', 900.3447494611517), ('well', 752.056532906337), ('opportunity', 745.3619653756549), ('personal', 733.1945564464819), ('flexible work', 730.9701692707621), ('get', 681.4878203353243), ('great', 678.3452051043357), ('work hour', 668.3499491005423), ('balance good', 635.0099999999384), ('timing', 632.0099999999082), ('onsite', 597.6120809407046), ('pressure', 577.1542662927604), ('bad', 575.9745706110365), ('ha', 536.918230591

[('learn', 15311.051600539104), ('lot', 7620.736357001958), ('part', 4953.2962426837275), ('good', 4933.84014458311), ('work', 4729.06276457899), ('opportunity', 4414.928527622445), ('job', 3852.916623090298), ('technology', 3671.4133617919), ('hard', 3519.434399409593), ('thing', 3517.2274704329348), ('skill', 2906.9030334215263), ('part job', 2629.6672843477922), ('enjoy', 2512.2873017565857), ('learn lot', 2462.0099999999106), ('wa', 2453.145502151616), ('many', 2417.045200926011), ('hard part', 2016.7020660595426), ('fresher', 1726.0318422146913), ('enjoy part', 1624.9153294450025), ('train', 1622.95992029784), ('time', 1520.393194832496), ('project', 1500.2766589598707), ('place', 1449.6641653915087), ('grow', 1198.6143347308744), ('start', 1090.967872304572), ('learn technology', 1082.009999999896), ('technical', 1079.351586883193), ('knowledge', 1076.9285402998685), ('got', 1049.2090030840059), ('also', 1030.6521786418873), ('provide', 994.656843049539), ('job wa', 940.857558055

In [107]:
aspect = {}
for idx, topic in enumerate(lda.components_):
    aspect['Aspect {0}'.format(str(idx))] = {vectorizer.get_feature_names()[i]: topic[i]
                                                         for i in topic.argsort()[:-100 - 1:-1]}

with open("../results/Sklearn-LDA/sklearn-lda-topic-k-7-a-0.01-b-0.01.json", "w") as f:
    json.dump(aspect, f)

In [111]:
from sklearn.decomposition import NMF
# Build a Non-Negative Matrix Factorization Model
nmf = NMF(n_components=7
                  ,init='nndsvd'
                 ,random_state=100,
                 alpha=0.01)

nmf.fit(X)


NMF(alpha=0.01, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
    max_iter=200, n_components=7, random_state=100, shuffle=False, solver='cd',
    tol=0.0001, verbose=0)

In [113]:

 
print("NMF Model:")
print_topics(nmf, vectorizer)
print("=" * 20)
 


NMF Model:
Topic 0:
[('work', 19.233540008011886), ('environment', 1.5748544207574555), ('culture', 1.399776131506253), ('co', 1.1850957426995008), ('co work', 1.1057213914593809), ('work environment', 1.0750316094514054), ('work culture', 1.0589827328594077), ('enjoy', 0.6692580504842008), ('time', 0.6314442834750299), ('hour', 0.6269220728039879), ('people', 0.6236705312604947), ('good work', 0.6055933531356917), ('project', 0.5610549959620448), ('fun', 0.5556198513717495), ('friendly', 0.5221540800046145), ('work place', 0.5202311985797462), ('flexible', 0.4908030042367164), ('year', 0.474638147128778), ('enjoy work', 0.44816094945561513), ('like', 0.4269480923614673), ('get', 0.41122377411467287), ('employee', 0.3958318947672869), ('typical', 0.3939464865343695), ('home', 0.3736151117191208), ('manager', 0.3381104042752783), ('work home', 0.3193401990944829), ('client', 0.31865421029652713), ('typical work', 0.307336476784194), ('fun work', 0.2972553554633137), ('would', 0.29346608

[('place', 10.262047881188604), ('great', 8.492104746372112), ('place work', 5.45925765208224), ('great place', 2.9035665311816063), ('good place', 1.9125352702235037), ('work place', 1.645598639631284), ('great work', 0.8984443176973695), ('work', 0.8277382670851133), ('people', 0.6739079267211815), ('fun', 0.6108347851510721), ('wa great', 0.5956078726040409), ('place learn', 0.5474959200157685), ('benefit', 0.5095656829002415), ('start', 0.4934873049916599), ('place culture', 0.4173197601587368), ('work great', 0.4125946785882047), ('place start', 0.34696537169408576), ('great people', 0.33770796382531026), ('fun place', 0.3247095208862713), ('great benefit', 0.28417449876555595), ('grow', 0.28103611176615767), ('overall', 0.27959435259156923), ('ha', 0.23316010128090225), ('opportunity', 0.22285714106149415), ('great experience', 0.20590964840252374), ('ha great', 0.1982269795029574), ('productive', 0.1944323933831911), ('excellent', 0.18532923751689415), ('great opportunity', 0.14

In [114]:
aspect = {}
for idx, topic in enumerate(nmf.components_):
    aspect['Aspect {0}'.format(str(idx))] = {vectorizer.get_feature_names()[i]: topic[i]
                                                         for i in topic.argsort()[:-100 - 1:-1]}

with open("../results/Sklearn-LDA/sklearn-nmf-topic-k-7-a-0.01.json", "w") as f:
    json.dump(aspect, f)